In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import time
import requests
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

/Users/estudo/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
url = 'https://www.transfermarkt.com.br/premier-league/gesamtspielplan/wettbewerb/GB1?saison_id=2024&spieltagVon=1&spieltagBis=38'
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'}

In [3]:
response = requests.get(url, headers = headers)
response

<Response [200]>

In [4]:
site = BeautifulSoup(response.text, 'html.parser')

In [5]:
# Coletar todos os resultados e nomes dos times
url = 'https://www.transfermarkt.com.br/premier-league/gesamtspielplan/wettbewerb/GB1?saison_id=2023&spieltagVon=1&spieltagBis=38'
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'}
site(url, headers = headers)

placar_list = []
time_casa_list = []
time_visita_list = []
#encontrando o placar
placar = site.find_all('td', class_ = 'zentriert hauptlink')
for item in placar:
    placar_list.append(item.get_text(strip=True))
#Time da casa
time_casa = site.find_all('td', class_ = 'text-right no-border-rechts hauptlink')
for item in time_casa:
    time_casa_list.append(item.get_text(strip=True))
#Time visitante
time_visita = site.find_all('td', class_ = 'no-border-links hauptlink')
for item in time_visita:
    time_visita_list.append(item.get_text(strip=True))
#DataFrame com os dados
resultado = pd.DataFrame({
        'Time_Casa': time_casa_list,
        'Placar': placar_list,
        'Time_Visita': time_visita_list})
  
resultado_tratado = resultado.apply(lambda col: col.str.replace(r'\(.*?\)', '', regex=True) if col.dtypes == 'object' else col)
resultado_tratado

,Time_Casa,Placar,Time_Visita
0,Man Utd,1:0,Fulham
1,Ipswich,0:2,Liverpool
2,Arsenal,2:0,Wolves
3,Everton,0:3,Brighton
4,Newcastle,1:0,Southampton
...,...,...,...
375,Newcastle,-:-,Everton
376,Nottm Forest,-:-,Chelsea
377,Southampton,-:-,Arsenal
378,Tottenham,-:-,Brighton


In [6]:
resultado_tratado[~resultado_tratado['Placar'].str.contains('-:-', case=False, na=False)]

,Time_Casa,Placar,Time_Visita
0,Man Utd,1:0,Fulham
1,Ipswich,0:2,Liverpool
2,Arsenal,2:0,Wolves
3,Everton,0:3,Brighton
4,Newcastle,1:0,Southampton
...,...,...,...
225,Man City,3:1,Chelsea
226,Crystal Palace,1:2,Brentford
227,Tottenham,1:2,Leicester
228,Aston Villa,1:1,West Ham


In [7]:
#coletar estatisticas da partida
session = requests.Session()
session.headers.update({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'})
lista_todos = []
url_base = 'https://www.transfermarkt.com.br'
url_acesso = site.find_all('a', 'ergebnis-link')
for link in url_acesso:
    url_jogo = link.get('href')
    url_stats = url_base+url_jogo.replace('index', 'statistik', 1)
    new_response = requests.get(url_stats, headers=headers)
    site_stats = BeautifulSoup(new_response.text, 'html.parser')
    stats_jogo = site_stats.find_all('div', class_ = 'sb-statistik-zahl')
    for x in stats_jogo:
        lista_stats = [x.get_text(strip=True) for x in stats_jogo]
    lista_todos.append(lista_stats)
  
    print(lista_stats)



['14', '10', '7', '4', '2', '4', '7', '8', '10', '11', '12', '10', '3', '1']
['7', '18', '2', '6', '3', '2', '2', '10', '17', '8', '9', '18', '5', '0']
['18', '9', '6', '5', '3', '4', '8', '2', '14', '17', '17', '14', '0', '1']
['9', '10', '4', '3', '2', '1', '1', '5', '8', '8', '8', '8', '7', '1']
['3', '19', '0', '4', '3', '0', '3', '12', '16', '15', '15', '16', '1', '2']
['14', '13', '3', '6', '3', '7', '2', '6', '8', '17', '17', '8', '1', '4']
['14', '15', '7', '6', '1', '3', '5', '3', '11', '18', '18', '11', '1', '1']
['9', '14', '3', '4', '6', '3', '4', '7', '15', '6', '6', '15', '1', '3']
['10', '11', '3', '3', '3', '3', '4', '3', '9', '12', '12', '9', '2', '1']
['7', '15', '3', '3', '5', '2', '2', '13', '12', '11', '11', '12', '2', '1']
['14', '11', '6', '5', '2', '2', '4', '4', '13', '8', '9', '13', '0', '6']
['14', '18', '8', '10', '1', '2', '3', '3', '16', '9', '9', '16', '3', '2']
['18', '10', '7', '4', '3', '4', '7', '5', '12', '14', '14', '13', '0', '5']
['14', '1', '6', 

In [8]:
resultado_tratado = resultado_tratado[~resultado_tratado['Placar'].str.contains('-:-', case=False, na=False)]

In [9]:
resultado_tratado[['Gols_Casa', 'Gols_Visita']] = resultado_tratado['Placar'].str.split(':', expand=True)

# Converter para inteiros, se necessário
resultado_tratado['Gols_Casa'] = resultado_tratado['Gols_Casa'].astype(int)
resultado_tratado['Gols_Visita'] = resultado_tratado['Gols_Visita'].astype(int)

/var/folders/m4/q93x5mln4l109zsf2n5yq6vr0000gn/T/ipykernel_3227/2925768477.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultado_tratado[['Gols_Casa', 'Gols_Visita']] = resultado_tratado['Placar'].str.split(':', expand=True)
/var/folders/m4/q93x5mln4l109zsf2n5yq6vr0000gn/T/ipykernel_3227/2925768477.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultado_tratado[['Gols_Casa', 'Gols_Visita']] = resultado_tratado['Placar'].str.split(':', expand=True)
/var/folders/m4/q93x5mln4l109zsf2n5yq6vr0000gn

In [10]:
dfstats_partida = pd.DataFrame(lista_todos, columns= ['Chute_casa','Chutes_visita','chute_errados_casa','chutes_errados_visita','defesas_casa','defesas_visita',
                                                           'escanteios_casa', 'escanteios_visita', 'faltas_sofridas_casa','faltas_sofridas_visita','faltas_cometidas_casa','faltas_cometidas_visita',
                                                           'impedimentos_casa', 'impedimentos_visita'])

In [15]:
resultado_tratado = resultado_tratado.dropna()

In [16]:
df_final = pd.concat([resultado_tratado,dfstats_partida], axis=1)
df_final

,Time_Casa,Placar,Time_Visita,Gols_Casa,Gols_Visita,Chute_casa,Chutes_visita,chute_errados_casa,chutes_errados_visita,defesas_casa,defesas_visita,escanteios_casa,escanteios_visita,faltas_sofridas_casa,faltas_sofridas_visita,faltas_cometidas_casa,faltas_cometidas_visita,impedimentos_casa,impedimentos_visita
0,Man Utd,1:0,Fulham,1.0,0.0,14,10,7,4,2,4,7,8,10,11,12,10,3,1
1,Ipswich,0:2,Liverpool,0.0,2.0,7,18,2,6,3,2,2,10,17,8,9,18,5,0
2,Arsenal,2:0,Wolves,2.0,0.0,18,9,6,5,3,4,8,2,14,17,17,14,0,1
3,Everton,0:3,Brighton,0.0,3.0,9,10,4,3,2,1,1,5,8,8,8,8,7,1
4,Newcastle,1:0,Southampton,1.0,0.0,3,19,0,4,3,0,3,12,16,15,15,16,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,Crystal Palace,1:2,Brentford,1.0,2.0,15,12,5,2,1,5,6,4,16,5,6,16,5,2
227,Tottenham,1:2,Leicester,1.0,2.0,14,14,6,6,2,3,4,3,14,11,13,16,2,4
228,Aston Villa,1:1,West Ham,1.0,1.0,9,4,6,2,0,2,3,0,9,7,7,9,2,2
229,Fulham,0:1,Man Utd,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df_final = df_final.dropna()

In [26]:
df_final.to_csv('/Users/estudo/Documents/Visualização-Dados-Premier-League/1_Bases_Originais/premier_league_24_25.csv', sep=';', index=False)